In [1]:
from secure_smtpd import SMTPServer, FakeCredentialValidator
from datetime import datetime
import os
import asyncore
import email
from datetime import datetime
import re,base64

class EmlHandler():
    def extract_image(self,data):
        
        image = None
        outf = None
        m = email.message_from_string(data)
        if m.is_multipart():
            for sm in m.get_payload():
                filename = sm.get_filename()
                if filename:
                    match = re.match(r"=\?UTF-8\?B\?(.*)\?=",str(filename))
                if filename and match:
                    filename = base64.b64decode(match.group(1)).decode('utf-8')
                if filename and filename.endswith('jpg'):
                    image = sm.get_payload(decode=True)
                elif filename:
                    print("Cannot handle {}".format(filename))
        if image:
            stamp = datetime.now()
            path = stamp.strftime("%Y%m%d")
            if not os.path.exists(path):
                os.makedirs(path)
            author = m['from'].replace('<','').split('@')[0]
            outf = "{}/{}-{}.jpg".format(path,stamp.strftime('%H%M%S'),author)
            with open(outf,'wb') as out:
                out.write(image)
        return outf

class EmlServer(SMTPServer):
    no = 0
    def process_message(self, peer, mailfrom, rcpttos, data,mail_options=None,rcpt_options=None):
        file = EmlHandler().extract_image(data)
        if file:
            print("Written image to {}".format(file))
        else:
            print("Mail from {} did not had picture inside it".format(mailfrom))

def run():
    foo = EmlServer(('192.168.0.241', 8025),
        None,
        require_authentication=False,
        ssl=False,
        credential_validator=FakeCredentialValidator(),
        )
    try:
        foo.run()
    except KeyboardInterrupt:
        pass

run()


OSError: [Errno 98] Address already in use